In [ ]:
import torch # to create tensors to store the raw data, weights and biases 
import torch.nn as nn # to make the weights and biases learnable (part of the network)
import torch.nn.functional as F # to apply activation functions 
from torch.optim import Adam  
from torch.distributions.uniform import Uniform

import lightning as L # to train the model 
from torch.utils.data import DataLoader, TensorDataset # to load the data 
from lightning.pytorch.tuner.tuning import Tuner      
from lightning.pytorch.callbacks import ModelCheckpoint #Lightning is trying to delete the previous checkpoint (because the default ModelCheckpoint is set to save_top_k=1).
                                                        # On Windows a file gets locked as soon as any program (Explorer preview, antivirus, TensorBoard, VS Code, …) opens it, and Windows then blocks the delete call ⇒ PermissionError WinError 32

import pandas as pd # to load the data 

import matplotlib.pyplot as plt # graphs 
import seaborn as sns # graphs


In [ ]:
inputs= torch.tensor([1,0,0,0],
                    [0,1,0,0],
                    [0,0,1,0],
                    [0,0,0,1])
labels= torch.tensor([0,1,0,0],
                    [0,0,1,0],
                    [0,0,0,1],
                    [0,1,0,0])

dataset= TensorDataset(inputs,labels)
dataloader= DataLoader(dataset)

In [1]:
class WordEmbeddingFromScratch(LightningModule):
    def __init__(self):
        super().__init__()

        min_value=-0.5
        max_value=0.5

        self.input1_w1 = nn.Parameter(Uniform(min_value, max_value).sample())
        self.input1_w2 = nn.Parameter(Uniform(min_value, max_value).sample())
        self.input2_w1 = nn.Parameter(Uniform(min_value, max_value).sample())
        self.input2_w2 = nn.Parameter(Uniform(min_value, max_value).sample())
        self.input3_w1 = nn.Parameter(Uniform(min_value, max_value).sample())
        self.input3_w2 = nn.Parameter(Uniform(min_value, max_value).sample())
        self.input4_w1 = nn.Parameter(Uniform(min_value, max_value).sample())
        self.input4_w2 = nn.Parameter(Uniform(min_value, max_value).sample())

        self.output1_w1 = nn.Parameter(Uniform(min_value, max_value).sample())
        self.output1_w2 = nn.Parameter(Uniform(min_value, max_value).sample())
        self.output2_w1 = nn.Parameter(Uniform(min_value, max_value).sample())
        self.output2_w2 = nn.Parameter(Uniform(min_value, max_value).sample())
        self.output3_w1 = nn.Parameter(Uniform(min_value, max_value).sample())
        self.output3_w2 = nn.Parameter(Uniform(min_value, max_value).sample())
        self.output4_w1 = nn.Parameter(Uniform(min_value, max_value).sample())
        self.output4_w2 = nn.Parameter(Uniform(min_value, max_value).sample())

        self.loss= nn.CrossEntropyLoss()

    def forward(self, input):
        input= input[0] # to remove the set of [] from the input list we get from the list 

        input_to_top_hidden = ((input[0] * self.input1_w1) +(input[1] * self.input2_w1) +(input[2] * self.input3_w1) +(input[3] * self.input4_w1))

        input_to_bottom_hidden = ((input[0] * self.input1_w2) +(input[1] * self.input2_w2) +(input[2] * self.input3_w2) +(input[3] * self.input4_w2))

        output1= (input_to_top_hidden * self.output1_w1) +(input_to_bottom_hidden * self.output1_w2)

        output2= (input_to_top_hidden * self.output2_w1) +(input_to_bottom_hidden * self.output2_w2)

        output3= (input_to_top_hidden * self.output3_w1) +(input_to_bottom_hidden * self.output3_w2)

        output4= (input_to_top_hidden * self.output4_w1) +(input_to_bottom_hidden * self.output4_w2)

        output_presoftmax = torch.stack([output1, output2, output3, output4], dim=1) # we use torch.stack to preserve the gradients of the output

        return output_presoftmax

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.1)

    def training_step(self, batch, batch_idx):
        input_i,label_i = batch
        output_i=self.forward(input_i)
        loss=self.loss(output_i, label_i[0])
        return loss



NameError: name 'LightningModule' is not defined